In [1]:
#data loading
import pandas as pd
predata = pd.read_csv(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Payments/Datasets/2payments_balanced_smote+rund-1M.csv', low_memory = False)

In [2]:
#-------------------------------------------------pre-processing--------------------------------------------------------------#

#begin of preprocessing
import time

start = time.time()

In [2]:
#convert amount and accountbalance to classes and assign a word to each interval  
import numpy as np

#automatic labels
import string


class LabelCategorizer:
    def __init__(self, base_word='cat'):
        self.initial = 1
        self._alphabet_index = 0
        self.base_word = base_word
        self.current_word = self.base_word
        self.shift = 0

    def __str__(self):
        return 'Class: Label Categorizer\nBase word: ' + self.base_word + '\nCurrent Word: ' + self.current_word

    def get_next_word(self):
        if self.initial:
            self.initial = 0
            return self.current_word

        if self.shift > 0:
            self.current_word = self.current_word[-1] + self.current_word[:-1]
            self.shift -= 1
        else:
            self.current_word = self.current_word + string.ascii_lowercase[self._alphabet_index]
            self._alphabet_index = (self._alphabet_index + 1) % len(string.ascii_lowercase)
            self.shift = len(self.current_word) - 1

        return self.current_word


#replacement of the old columns with the new ones with classes
def cutter(col, number, word, words_map):
    #make sure that only positives are assigned an interval
    col_min = max(predata[col].min(), 1)
    col_max = max(predata[col].max(), 1)

    bins_a = np.geomspace(float(col_min), float(col_max), num=number)
    bins_a[0] = bins_a[0] - 1
    bins_aux = bins_a[1:]
    bins_aux = np.append(bins_aux, bins_a[-1] + 1)
    bin_tuples = list(zip(bins_a, bins_aux))

    bins = pd.IntervalIndex.from_tuples(bin_tuples)

    #range of the intervals made
    labels_a = []

    a = LabelCategorizer(base_word=word)

    for _ in range(number):
        labels_a.append(a.get_next_word())

    x = pd.cut(predata[col].to_list(), bins=bins)
    x.categories = labels_a
    predata[col] = x

    for i in range(number):
        words_map[labels_a[i]] = bins[i]

    #columns to apply the conversion


columns = ['amount', 'accountbalance']

#number of intervals for each column
number_bins = [37, 38]

#base words assigned to each column on columns to apply the conversion
base_words = ['pink', 'red']

#get acess to the range of the interval based on the word that appears
values_map = {}

for i in range(len(columns)):
    cutter(columns[i], number_bins[i], base_words[i], values_map)

In [4]:
#bins amount
#transfers: 10m - 25.123 / 1M - 35.8775 / 2M - 37.3285 / 3M - 38.2194 / 4M - 38.818 / 5M - 39.3326 / gans - 26.3159 / imbalanced - 24.7233
#payments: 10m -  / 1M - 36.7452 / 2M - 38.2134 / 3M - 39.1709 / 4M - 39.8368 / 5M - 40.2234 / gans - 24.8089 / imbalanced - 22.5533

#bins accountbalance
#transfers: 10m - 24.6482 / 1M - 37.0555 / 2M - 38.5003 / 3M - 39.4439 / 4M - 40.1224 / 5M - 40.627 / gans - 25.734 / imbalanced - 23.9742
#payments: 10m - / 1M - 37.234 / 2M - 38.6645 / 3M - 39.651 / 4M - 40.2506 / 5M - 40.7402 / gans - 26.0935 / imbalanced - 20.7986

In [3]:
#assign the word negaccount for negative values of accountbalance  
aux = predata['accountbalance'].values

vacc = []

for elm in aux:
    if str(elm) == 'nan':
        vacc.append('negaccount')
    else:
        vacc.append(elm)
predata['accountbalance'] = vacc

In [9]:
#interval that a word corresponds to
#values_map['apbicndke']

Interval(34055.72399772817, 50124.37964687015, closed='right')

In [4]:
#convert hours to classes and assign a word to each interval  
bins_hour = [0, 4, 8, 12, 16, 20, 24]

#labels assigned to each interval
labels_hour = ['dawn', 'earlymorning', 'morning', 'afternoon', 'dusk', 'night']

predata['hour'] = list(pd.cut(predata['hour'], bins=bins_hour, labels=labels_hour, retbins=True, include_lowest=True)[0])

In [5]:
#map trusted_indicator values to words
ti_value_map = {
    '0.0': 'ntrusted',
    '0': 'ntrusted',
    '1.0': 'trusted',
    'unknown': 'tunknown'
}

predata['trusted_indicator'] = predata['trusted_indicator'].apply(lambda x: ti_value_map[x])

In [6]:
#add letter before number to distinguish between similar numbers from different columns
cols = ['entity', 'reference', 'iban_orig', 'iban_dest', 'ipaddress', 'clientid', 'week']

identifier = ['e', 'r', 'io', 'id', 'ip', 'c', 'w']

for col in range(len(cols)):
    predata[cols[col]] = predata[cols[col]].apply(lambda x: identifier[col] + str(x))

In [7]:
#convert binary and chains of numbers to specific words
def apply_map(df, col, target_map):
    df[col] = df[col].apply(lambda x: target_map.get(str(x)))


cols_maps = [('is_fraud', {'0': 'nfraud', '1': 'fraud'}),
             ('weekday', {'0': 'mon', '1': 'tue', '2': 'wed', '3': 'thu', '4': 'fri', '5': 'sat', '6': 'sun'}),
             ('month', {'1': 'jan', '2': 'feb', '3': 'mar', '4': 'apr', '5': 'may', '6': 'jun', '7': 'jul', '8': 'aug', 
                        '9': 'sep', '10': 'oct', '11': 'nov', '12': 'dec'})]

for comb in cols_maps:
    apply_map(predata, comb[0], comb[1])

In [8]:
#make fraud column as the center column
new_order = ['canal', 'operativa', 'clientid', 'entity', 'reference', 'trusted_indicator', 'iban_orig', 'iban_dest',
             'amount', 'is_fraud', 'accountbalance', 'ipaddress', 'browser_family', 'os_family', 'hour', 'week', 'weekday',
             'month', 'device']

predata = predata[new_order]

In [9]:
#select data for train and test  
from sklearn.model_selection import train_test_split

#target column
y = predata['is_fraud']
predata.drop('is_fraud', axis=1)

#train and test
X_train, X_test, y_train, y_test = train_test_split(predata, y, stratify=y, test_size=0.3)

In [10]:
#function for conversion format
def convert_to_w2v_format(df):
    sentences = df.to_numpy()
    sentences_aux = [list(curr) for curr in sentences]
    sentences_series = pd.Series(sentences_aux)
    return sentences_series

In [11]:
#for entering the model
sentences_series = convert_to_w2v_format(X_train)

In [12]:
#for the test metrics
X_test_np = X_test.copy()
sentences_series_np_test = convert_to_w2v_format(X_test_np)

In [13]:
#for the train metrics
X_train_np = X_train.copy()
sentences_series_np_train = convert_to_w2v_format(X_train_np)

In [14]:
#get size of the corpus 
token_count = sum([len(sentence) for sentence in sentences_series])

print("This corpus contains {} tokens".format(token_count))

This corpus contains 13300000 tokens


In [15]:
#end of preprocessing
stop = time.time()

print(f"Pre-Processing: {stop - start}s")

Training time: 43.16966795921326s


In [17]:
#-----------------------------------------------------training----------------------------------------------------------------#

#begin of training
begin = time.time()

In [14]:
#inicialization and training word2vec 
import multiprocessing
from myWord2Vec import MyWord2Vec
from gensim.models.callbacks import CallbackAny2Vec

def training(sentences, dim, sample, negative, alpha, min_alpha):
    model = MyWord2Vec(
        sg = 1,  #skip-gram 
        workers = multiprocessing.cpu_count(),  #use all cores 
        vector_size = dim,  #dimension of the embedding space 
        window = 1,  #words befores and after the center word 
        sample = sample,  #whithout subsampling 
        min_count = 1,  #use every word 
        negative = negative,  #noise-words 
        hs = 0,  #negative sampling
        ns_exponent = 0.75,  #exponent to shape negative sampling 
        alpha = alpha,  #initial learning rate 
        min_alpha = min_alpha,  #final learning rate 
    )

    #vocabulary creation
    model.build_vocab(sentences)

    #model training
    model.train(sentences, epochs = 150, total_examples = model.corpus_count, compute_loss = True, callbacks = [])

    return model

In [19]:
#end of training
end = time.time()

print(f"Training: {end - begin}s")

Training: 6.508042335510254s


In [24]:
#-----------------------------------------------------prediction-------------------------------------------------------------#

#begin of test - 1st step
begin = time.time()

In [15]:
#update probv - vector with the parameters of a transaction
from scipy.special import softmax

def predictTransaction(my_model, m_cid, my_curr_cid, verbose):
    #obtain the representative vector of the clientid
    civ = my_model.wv.get_vector(m_cid)

    #multiplication of the clientid vector by the decode matrix (M2)  
    m2 = my_model.syn1neg
    
    #vector of len = vocab_size that softmax is applied to obtain the conditional probabilities 
    vout = softmax(np.dot(civ, m2.T))

    fraud_curr = my_curr_cid.copy()
    fraud_curr.append(my_model.wv.index_to_key.index('fraud'))

    nfraud_curr = my_curr_cid.copy()
    nfraud_curr.append(my_model.wv.index_to_key.index('nfraud'))

    #filter the vector to the transaction fields
    fraud_probv = sum(vout[fraud_curr])
    nfraud_probv = sum(vout[nfraud_curr])

    #print results if verbose True
    if verbose:
        print(f"Transaction: {my_curr_cid}\nFraud prob: {fraud_probv}\nNon Fraud prob: {nfraud_probv}")

    #threshold for false negatives
    # if fraud_probv > threshold: return 1

    if fraud_probv > nfraud_probv:
        return 1
    
    return 0

In [16]:
#prediction method   
def predict(model, X, verbose1, verbose2):
    #save the predictions made by the model in a list
    predictions = []

    #for each transaction (eval_row)
    for i, eval_row in enumerate(X, 1):

        #print transaction parameters if verbose1 True
        if verbose1:
            print(eval_row)

        #cid is always in position 2 of the array
        curr_cid = eval_row[2]

        #for each sentence_series creates a current list
        curr = []

        #if curr_cid is not known
        if curr_cid not in model.wv.index_to_key:
            #update model
            model.build_vocab([[curr_cid]], update=True)

        #filter the parameters associated with the transaction made by the specific clientid (civ)
        for x in eval_row:
            if x != curr_cid:
                if x in model.wv.index_to_key:
                    curr.append(model.wv.index_to_key.index(x))

                else:
                    #if the word is not known                        
                    #update model
                    model.build_vocab([[x]], update=True)
                    curr.append(model.wv.index_to_key.index(x))

        #sum each value on the array to obtain the final probability
        curr_prediction = predictTransaction(model, curr_cid, curr, verbose2)
        predictions.append(curr_prediction)

    return predictions

In [ ]:
#end of test - 1st step
end = time.time()

print(f"1st Step Test: {end - begin}s")

In [17]:
#convert true values for the train set to binary
y_train = [1 if elem == "fraud" else 0 for elem in y_train]

In [18]:
#convert true values of the test set to binary
y_test = [1 if elem == "fraud" else 0 for elem in y_test]

In [19]:
#grid-search
import os
from mlflow import log_metric, log_param, log_artifact
import mlflow
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix

#create a hyperparameters dictionary 
dim = [2, 5]   
negative = [5, 10] #0 leads to not having the syn1neg property
alpha = [0.025, 0.035] 
min_alpha = [0.0001, 0.001] 
sample = [0, 0.001, 0.01]

#train the model 
i = 0
for d in dim:
    for n in negative:
        for a in alpha:
            for m in min_alpha:
                for s in sample:
                    i+=1
                    with mlflow.start_run(nested=True):

                        # Log params to mlflow
                        log_param("dim", d)
                        log_param("negative", n)
                        log_param("alpha", a)
                        log_param("min_alpha", m)
                        log_param("sample", s)

                        curr_model = training(sentences_series_np_train, dim = d, sample = s, negative = n, alpha = a, min_alpha = m)

                        #metrics for each combination - train
                        prevs_train = predict(curr_model, sentences_series_np_train[:10000], verbose1 = False, verbose2 = False)
                        precision_train, recall_train, fscore_train, _ = precision_recall_fscore_support(y_train[:10000], prevs_train, average='micro')
                        tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train[:10000], prevs_train, labels = [0, 1]).ravel()
                        
                        log_param("accuracy_train", accuracy_score(y_train[:10000], prevs_train))
                        log_param("precision_train", precision_train)
                        log_param("recall_train", recall_train)
                        log_param("f1_train", fscore_train)
                        log_param("tn_train", tn_train)
                        log_param("fp_train", fp_train)
                        log_param("fn_train", fn_train)
                        log_param("tp_train", tp_train)

                        #save current model
                        curr_model.save("trained_model.w2v")
                        log_artifact("trained_model.w2v")
                        os.remove("trained_model.w2v")

Validation loss decreased (inf --> 6661974.000000).  Saving model ...
Validation loss decreased (6661974.000000 --> 5827733.000000).  Saving model ...
Validation loss decreased (5827733.000000 --> 5074643.000000).  Saving model ...
Validation loss decreased (5074643.000000 --> 2729042.000000).  Saving model ...
Validation loss decreased (2729042.000000 --> 2692026.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (2692026.000000 --> 2582870.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (2582870.000000 --> 1315980.000000).  Saving model ...
Validation loss decreased (1315980.000000 --> 1286320.000000).  Saving model ...
Validation loss decreased (1286320.000000 --> 1283096.000000).  Saving model ...
Validation loss decreased (1283096.000000 --> 1249076.000000).  Saving model ...
Validation loss decreased (1249076.000000 --> 1228644.000000).  Saving model ...
Validation loss

EarlyStopping counter: 9 out of 25
EarlyStopping counter: 10 out of 25
EarlyStopping counter: 11 out of 25
EarlyStopping counter: 12 out of 25
EarlyStopping counter: 13 out of 25
EarlyStopping counter: 14 out of 25
Validation loss decreased (1144660.000000 --> 275180.000000).  Saving model ...
Validation loss decreased (275180.000000 --> 17144.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
EarlyStopping counter: 8 out of 25
EarlyStopping counter: 9 out of 25
EarlyStopping counter: 10 out of 25
EarlyStopping counter: 11 out of 25
EarlyStopping counter: 12 out of 25
EarlyStopping counter: 13 out of 25
EarlyStopping counter: 14 out of 25
EarlyStopping counter: 15 out of 25
EarlyStopping counter: 16 out of 25
EarlyStopping counter: 17 out of 25
EarlyStopping counter: 1

EarlyStopping counter: 24 out of 25
EarlyStopping counter: 25 out of 25
Validation loss decreased (inf --> 5086396.000000).  Saving model ...
Validation loss decreased (5086396.000000 --> 4216969.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (4216969.000000 --> 3337451.000000).  Saving model ...
Validation loss decreased (3337451.000000 --> 1667478.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
EarlyStopping counter: 8 out of 25
Validation loss decreased (1667478.000000 --> 829156.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopp

EarlyStopping counter: 8 out of 25
EarlyStopping counter: 9 out of 25
EarlyStopping counter: 10 out of 25
Validation loss decreased (5328.000000 --> 5288.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
Validation loss decreased (5288.000000 --> 4824.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (4824.000000 --> 4696.000000).  Saving model ...
Validation loss decreased (4696.000000 --> 4672.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
Validation loss decreased (4672.000000 --> 4464.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out 

EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
Validation loss decreased (1428824.000000 --> 1422772.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
Validation loss decreased (1422772.000000 --> 1417556.000000).  Saving model ...
Validation loss decreased (1417556.000000 --> 1398800.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
Validation loss decreased (1398800.000000 --> 1380280.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
Validation loss decreased (1380280.000000 --> 1340320.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (1340320.000000 --> 152496.000000).  Saving 

...

Validation loss decreased (87336.000000 --> 83240.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (83240.000000 --> 82464.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (82464.000000 --> 79936.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (79936.000000 --> 78800.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
Validation loss decreased (78800.000000 --> 77248.000000).  Saving model ...
Validation loss decreased (77248.000000 --> 76136.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
Validation loss decreased (76136.000000 --> 76104.000000).  Saving model ...
Validation loss decreased (76104.000000 -

Validation loss decreased (3923762.000000 --> 2203267.000000).  Saving model ...
Validation loss decreased (2203267.000000 --> 1991894.000000).  Saving model ...
Validation loss decreased (1991894.000000 --> 1945302.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (1945302.000000 --> 1926472.000000).  Saving model ...
Validation loss decreased (1926472.000000 --> 1905676.000000).  Saving model ...
Validation loss decreased (1905676.000000 --> 1888154.000000).  Saving model ...
Validation loss decreased (1888154.000000 --> 1866728.000000).  Saving model ...
Validation loss decreased (1866728.000000 --> 1847132.000000).  Saving model ...
Validation loss decreased (1847132.000000 --> 1425598.000000).  Saving model ...
Validation loss decreased (1425598.000000 --> 1330672.000000).  Saving model ...
Validation loss decreased (1330672.000000 --> 1307560.000000).  Saving model ...
Validation loss decreased (1307560.000000 --> 1272648.000000).  Saving mod

EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
Validation loss decreased (124888.000000 --> 122520.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
EarlyStopping counter: 8 out of 25
EarlyStopping counter: 9 out of 25
EarlyStopping counter: 10 out of 25
EarlyStopping counter: 11 out of 25
EarlyStopping counter: 12 out of 25
EarlyStopping counter: 13 out of 25
EarlyStopping counter: 14 out of 25
EarlyStopping counter: 15 out of 25
EarlyStopping counter: 16 out of 25
EarlyStopping counter: 17 out of 25
EarlyStopping counter: 18 out of 25
EarlyStopping counter: 19 out of 25
EarlyStopping counter: 20 out of 25
EarlyStopping counter: 21 out of 25
Validation loss decreased (inf --> 6450543.500000).  Saving model ...
Validation loss decreased (6450543

Validation loss decreased (55048.000000 --> 54864.000000).  Saving model ...
Validation loss decreased (54864.000000 --> 54600.000000).  Saving model ...
Validation loss decreased (54600.000000 --> 54344.000000).  Saving model ...
Validation loss decreased (54344.000000 --> 53784.000000).  Saving model ...
Validation loss decreased (53784.000000 --> 53464.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (53464.000000 --> 52864.000000).  Saving model ...
Validation loss decreased (52864.000000 --> 50584.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (50584.000000 --> 49592.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
Validation loss decreased (49592.000000 --> 47176.000000).  Saving model ...
Validation loss decreased (47176.000000 --> 46312.000000).  Saving model ...
EarlyStopping counte

Validation loss decreased (1089292.000000 --> 1084172.000000).  Saving model ...
Validation loss decreased (1084172.000000 --> 1076876.000000).  Saving model ...
Validation loss decreased (1076876.000000 --> 1062632.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (1062632.000000 --> 1050408.000000).  Saving model ...
Validation loss decreased (1050408.000000 --> 1046404.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (1046404.000000 --> 835880.000000).  Saving model ...
Validation loss decreased (835880.000000 --> 197280.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
Validation loss decreased (197280.000000 --> 196080.000000).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (196080.000000 --> 191408.000000).  Saving model ...
Validation loss decreased (191408.000000 --> 187320.000000).  Saving

In [ ]:
#apply the prediction method for the train set
#predict(model, X_train_np.values, verbose1=True, verbose2=True)

In [ ]:
#true values for the train set
#y_train

In [ ]:
#metrics for the train set
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, matthews_corrcoef, confusion_matrix, precision_recall_curve

trs = y_train
prevs = predict(model, X_train_np.values, verbose1 = False, verbose2 = False, shouldBe = trs)

#accuracy
accuracy = accuracy_score(trs,prevs)
print('accuracy: {}'.format(accuracy))

#precision, recall, f-score
precision, recall, fscore, support = precision_recall_fscore_support(trs, prevs, average = 'micro')

#precision - ratio tp / (tp + fp) - ability not to label a negative sample as positive
print('precision: {}'.format(precision))

#recall - ratio tp / (tp + fn) - ability to find all the positive samples - best is 1, worst is 0
print('recall: {}'.format(recall))

#fscore - weighted harmonic mean of the precision and recall - best is 1, worst is 0
print('fscore: {}'.format(fscore))

#matthews correlation coefficient - measure of the quality of binary classifications
#can be used even if the classes are of very different sizes - is in essence a correlation coefficient between -1 and +1
#+1 means perfect prediction, 0 an average random prediction, -1 an inverse prediction
mcc = matthews_corrcoef(trs, prevs)
print('mcc: {}'.format(mcc))

#g-mean - squared root of the product of the sensitivity and specificity - best is 1, worst is 0
print('G-mean:', (geometric_mean_score(trs, prevs, average = 'micro')))

#sensitivity - ability to predict true positives of each available category = recall

#confusion matrix    
print(confusion_matrix(trs, prevs))

#true positives, false positives, true negatives, false negatives
tn, fp, fn, tp = confusion_matrix(trs,prevs).ravel()

#true negatives
print('true negatives: {}'.format(tn))

#false positives
print('false positives: {}'.format(fp))

#false negatives
print('false negatives: {}'.format(fn))

#false positives
print('true positives: {}'.format(tp))

#specificity - ability to predict true negatives of each available category - recall of the negative class
specifity = tn / (tn + fp)
print('specifity: {}'.format(specifity))

#roc curve
import matplotlib.pyplot as plt
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(trs,prevs)

roc_auc = metrics.auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr = fpr, tpr = tpr, roc_auc = roc_auc, estimator_name = 'Word2vec')
display.plot()
plt.show()

In [ ]:
#apply the prediction method for the test set
#predict(model, X_test_np.values, verbose1=True, verbose2=True)

In [ ]:
#true values of the test set
#set(y_test)

In [20]:
#load the ideal model
import gensim.models.word2vec as w2v

model = w2v.Word2Vec.load(r'C:/Users/BeatrizCarvalho/Downloads/payments-1M/trained_model.w2v')

In [42]:
#begin of test - 2nd step
import time

begin = time.time()

In [21]:
#metrics for the test set 
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, matthews_corrcoef, confusion_matrix, precision_recall_curve

trs = y_test[:10000]
prevs = predict(model, X_test_np.values[:10000], verbose1 = False, verbose2 = True)

#accuracy
accuracy = accuracy_score(trs, prevs)
print('accuracy: {}'.format(accuracy))

#precision, recall, f-score
precision, recall, fscore, support = precision_recall_fscore_support(trs, prevs, average = 'micro')

#precision - ratio tp / (tp + fp) - ability not to label a negative sample as positive
print('precision: {}'.format(precision))

#recall - ratio tp / (tp + fn) - ability to find all the positive samples - best is 1, worst is 0
print('recall: {}'.format(recall))

#fscore - weighted harmonic mean of the precision and recall - best is 1, worst is 0
print('fscore: {}'.format(fscore))

#matthews correlation coefficient - measure of the quality of binary classifications
#can be used even if the classes are of very different sizes - is in essence a correlation coefficient between -1 and +1
#+1 means perfect prediction, 0 an average random prediction, -1 an inverse prediction
mcc = matthews_corrcoef(trs, prevs)
print('mcc: {}'.format(mcc))

#g-mean - squared root of the product of the sensitivity and specificity - best is 1, worst is 0
print('G-mean:', (geometric_mean_score(trs, prevs, average = 'micro')))

#sensitivity - ability to predict true positives of each available category = recall

#confusion matrix    
print(confusion_matrix(trs, prevs))

#true positives, false positives, true negatives, false negatives
tn, fp, fn, tp = confusion_matrix(trs, prevs).ravel()

#true negatives
print('true negatives: {}'.format(tn))

#false positives
print('false positives: {}'.format(fp))

#false negatives
print('false negatives: {}'.format(fn))

#false positives
print('true positives: {}'.format(tp))

#specificity - ability to predict true negatives of each available category - recall of the negative class
specifity = tn / (tn + fp)
print('specifity: {}'.format(specifity))

#precision-recall curve - compute precision-recall pairs for different probability thresholds
#print(precision_recall_curve(trs, prevs))

#roc curve
import matplotlib.pyplot as plt
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(trs, prevs)

roc_auc = metrics.auc(fpr, tpr)

display = metrics.RocCurveDisplay(fpr = fpr, tpr = tpr, roc_auc = roc_auc, estimator_name = 'Word2vec')
display.plot()
plt.show()

Transaction: [9, 2, 25, 204, 0, 56, 1, 35, 7, 27, 148, 14, 12, 5, 37, 21, 20, 15]
Fraud prob: 0.9834044762375711
Non Fraud prob: 0.9834044762375754
Transaction: [16, 44, 215, 1785066, 0, 1785067, 1, 43, 8, 70, 1785068, 29, 87, 80, 95, 58, 45, 3]
Fraud prob: 2.1822061043508256e-05
Non Fraud prob: 2.1796436030285804e-05
Transaction: [9, 2, 8226, 1785070, 0, 1785071, 1, 64, 7, 41, 1785072, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 8.774977487746582e-06
Non Fraud prob: 8.798838706525203e-06
Transaction: [17, 2, 164, 1785073, 0, 860977, 1, 30, 8, 38, 1785074, 75, 4, 18, 84, 26, 32, 3]
Fraud prob: 5.2124533622077696e-05
Non Fraud prob: 5.215642687278477e-05
Transaction: [16, 2, 7855, 1785076, 0, 1785077, 1785078, 76, 7, 39, 1785079, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 9.481358901552994e-06
Non Fraud prob: 9.49290405571901e-06
Transaction: [9, 2, 25, 144, 0, 56, 1, 30, 7, 27, 110, 14, 12, 5, 37, 58, 20, 15]
Fraud prob: 0.9834044162539438
Non Fraud prob: 0.9834044162539481
Transaction: [9, 2, 25, 17

Transaction: [16, 44, 272, 1785173, 71, 252043, 1, 43, 8, 23, 1689413, 6, 4, 5, 149, 10, 40, 3]
Fraud prob: 3.5035552309438736e-05
Non Fraud prob: 3.5104897940898064e-05
Transaction: [9, 2, 99, 1785175, 0, 1785176, 1, 30, 8, 70, 1785177, 14, 12, 80, 137, 31, 53, 15]
Fraud prob: 1.4872175597702153e-05
Non Fraud prob: 1.4941607446417038e-05
Transaction: [9, 2, 1355, 1785179, 0, 1785180, 1, 30, 8, 23, 1785181, 29, 42, 5, 124, 22, 51, 34]
Fraud prob: 1.1944642437811126e-05
Non Fraud prob: 1.1819754888620082e-05
Transaction: [17, 2, 69, 338, 0, 336, 1, 49, 7, 36, 337, 6, 4, 5, 126, 21, 32, 3]
Fraud prob: 0.26340875180619194
Non Fraud prob: 0.2634087518061919
Transaction: [17, 2, 3963, 1785183, 0, 161, 1, 19, 7, 27, 140, 65, 4, 18, 13, 10, 11, 3]
Fraud prob: 1.3357506702504907e-05
Non Fraud prob: 1.3312026624134887e-05
Transaction: [9, 2, 5576, 1785185, 0, 1785186, 1, 49, 7, 73, 1785187, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 8.256396746730843e-06
Non Fraud prob: 8.248134925281647e-06
Transacti

Transaction: [16, 2, 92, 1785271, 0, 1785272, 1, 74, 7, 38, 731, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 1.0330147318882155e-05
Non Fraud prob: 1.0373974902222471e-05
Transaction: [17, 2, 28, 122, 0, 431, 1, 19, 7, 88, 430, 6, 4, 5, 59, 21, 33, 3]
Fraud prob: 0.0017683154121609698
Non Fraud prob: 0.001768315412160969
Transaction: [9, 44, 66, 1785274, 0, 1785275, 1, 43, 8, 38, 1785276, 14, 12, 5, 109, 31, 32, 15]
Fraud prob: 1.2676710817061121e-05
Non Fraud prob: 1.2727591190753174e-05
Transaction: [17, 2, 164, 1785277, 0, 1613513, 1, 63, 8, 38, 1613514, 6, 4, 47, 183, 31, 93, 3]
Fraud prob: 1.571564158409089e-05
Non Fraud prob: 1.5637184542338645e-05
Transaction: [16, 2, 234, 1785279, 0, 1785280, 1, 81, 8, 23, 1785281, 75, 4, 5, 210, 21, 93, 3]
Fraud prob: 1.5368365495760372e-05
Non Fraud prob: 1.544433638400733e-05
Transaction: [9, 2, 510, 1785283, 0, 1785284, 1785285, 76, 7, 36, 1785286, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 1.142982799251513e-05
Non Fraud prob: 1.1454816586820016e-05
Tran

Transaction: [16, 2, 446, 1785384, 0, 1785385, 1, 81, 8, 23, 1775501, 29, 87, 18, 121, 26, 11, 3]
Fraud prob: 9.661192024168486e-06
Non Fraud prob: 9.677133107288682e-06
Transaction: [16, 44, 736, 1785387, 71, 1785388, 1, 60, 8, 36, 1785389, 6, 4, 47, 154, 21, 53, 3]
Fraud prob: 8.362468150835412e-06
Non Fraud prob: 8.377565649198004e-06
Transaction: [17, 2, 69, 1785391, 0, 1785392, 1, 49, 7, 38, 1785393, 6, 4, 5, 13, 21, 32, 3]
Fraud prob: 1.1926868126010959e-05
Non Fraud prob: 1.1922414842047147e-05
Transaction: [9, 2, 163, 1785395, 0, 1785396, 1, 43, 8, 38, 97845, 14, 12, 47, 154, 58, 53, 15]
Fraud prob: 1.0958764448787406e-05
Non Fraud prob: 1.0918730822595535e-05
Transaction: [9, 2, 206, 1785398, 0, 1785399, 1, 57, 8, 23, 1785400, 14, 12, 24, 116, 21, 51, 15]
Fraud prob: 1.4521053600446976e-05
Non Fraud prob: 1.4590799963798418e-05
Transaction: [9, 44, 499, 1785402, 0, 1785403, 1, 43, 8, 23, 1785404, 29, 42, 5, 86, 22, 45, 34]
Fraud prob: 1.6576111960375783e-05
Non Fraud prob: 1.6

Transaction: [17, 2, 164, 1785519, 0, 3763, 1, 35, 8, 88, 9725, 75, 4, 18, 149, 22, 40, 3]
Fraud prob: 0.0021095633670145013
Non Fraud prob: 0.0021095625650088727
Transaction: [16, 2, 484, 1785520, 0, 134983, 1, 78, 8, 41, 92667, 6, 4, 18, 95, 21, 45, 3]
Fraud prob: 2.3605920138572434e-05
Non Fraud prob: 2.3570276382181565e-05
Transaction: [9, 2, 6887, 1785522, 0, 1785523, 1, 30, 7, 38, 49483, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 1.2163910497520192e-05
Non Fraud prob: 1.2164430671646187e-05
Transaction: [17, 2, 164, 1785524, 0, 951335, 1, 30, 8, 120, 951325, 6, 87, 24, 134, 31, 53, 3]
Fraud prob: 0.00024613610092750343
Non Fraud prob: 0.00024613032950149966
Transaction: [9, 2, 11924, 1785526, 0, 188679, 1, 49, 7, 48, 1785527, 6, 4, 5, 13, 10, 32, 3]
Fraud prob: 9.129502089422203e-06
Non Fraud prob: 9.15770412746042e-06
Transaction: [9, 2, 3363, 1785529, 0, 1785530, 1, 82, 7, 36, 1785531, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 9.912322951777242e-06
Non Fraud prob: 9.894006410604561e-06
Tran

...

Transaction: [9, 2, 322, 1790591, 0, 282182, 1, 68, 8, 50, 89844, 14, 12, 18, 142, 55, 61, 15]
Fraud prob: 4.089859592829037e-05
Non Fraud prob: 4.093595654955373e-05
Transaction: [16, 107, 52, 100, 0, 175, 179, 62, 7, 89, 595, 29, 87, 80, 104, 21, 11, 3]
Fraud prob: 0.0006140791072333229
Non Fraud prob: 0.0006140791072333229
Transaction: [17, 2, 69, 1790593, 0, 1790594, 1, 76, 7, 41, 1790595, 6, 4, 5, 13, 10, 32, 3]
Fraud prob: 9.592527170809717e-06
Non Fraud prob: 9.621545849824997e-06
Transaction: [9, 44, 181, 1046986, 0, 119687, 1, 57, 8, 180, 1790596, 29, 42, 18, 95, 22, 45, 34]
Fraud prob: 0.00022397988700215187
Non Fraud prob: 0.00022399901465627913
Transaction: [16, 2, 131, 1790598, 71, 1790599, 1, 30, 8, 39, 1790600, 75, 4, 5, 128, 26, 53, 3]
Fraud prob: 7.727649695254968e-06
Non Fraud prob: 7.724375343798329e-06
Transaction: [17, 2, 448, 1790601, 0, 277805, 1, 35, 8, 120, 175153, 75, 4, 5, 154, 55, 53, 3]
Fraud prob: 1.7797598955870964e-05
Non Fraud prob: 1.7813171302838438e-

Transaction: [9, 2, 25, 1790676, 0, 91, 1, 30, 7, 23, 808, 14, 12, 5, 37, 10, 20, 15]
Fraud prob: 0.9833950083850881
Non Fraud prob: 0.9833950083850924
Transaction: [9, 2, 458, 1790678, 0, 1790679, 1, 82, 8, 27, 1790680, 14, 12, 94, 139, 10, 45, 15]
Fraud prob: 1.0488994774959792e-05
Non Fraud prob: 1.048933322067569e-05
Transaction: [9, 2, 25, 204, 0, 56, 1, 35, 7, 27, 148, 14, 12, 5, 37, 21, 20, 15]
Fraud prob: 0.9833951183053219
Non Fraud prob: 0.9833951183053262
Transaction: [16, 2, 99, 1790681, 71, 1667827, 1, 57, 8, 36, 9609, 6, 4, 24, 134, 22, 53, 3]
Fraud prob: 0.00022949519643805302
Non Fraud prob: 0.00022960273939747963
Transaction: [9, 2, 25, 1467275, 0, 77, 1, 43, 7, 50, 79, 14, 12, 5, 72, 10, 20, 15]
Fraud prob: 0.9803928721410533
Non Fraud prob: 0.9803928721410754
Transaction: [9, 2, 478, 1790683, 0, 1790684, 1790685, 76, 7, 39, 1790686, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 1.0943473142788207e-05
Non Fraud prob: 1.0878739914232938e-05
Transaction: [17, 2, 164, 1790687, 0, 

Transaction: [9, 2, 96, 1790773, 0, 1790774, 1, 35, 8, 39, 1790775, 14, 12, 94, 138, 26, 53, 15]
Fraud prob: 1.2734264117852945e-05
Non Fraud prob: 1.2755716255696825e-05
Transaction: [9, 2, 659, 1790777, 0, 1790778, 1, 60, 8, 70, 1790779, 14, 12, 5, 135, 31, 61, 15]
Fraud prob: 1.3361327887650987e-05
Non Fraud prob: 1.3329752960089536e-05
Transaction: [16, 44, 272, 1790780, 0, 189659, 1, 30, 8, 67, 1790781, 14, 4, 5, 127, 21, 33, 174]
Fraud prob: 4.999275663577574e-05
Non Fraud prob: 5.001821563779341e-05
Transaction: [9, 2, 4787, 1790783, 0, 1790784, 1, 49, 7, 41, 1655090, 6, 4, 5, 13, 22, 11, 3]
Fraud prob: 9.828649865539774e-06
Non Fraud prob: 9.89695445241523e-06
Transaction: [16, 2, 172, 1790786, 0, 562341, 1, 60, 8, 41, 1790787, 6, 4, 24, 121, 10, 11, 3]
Fraud prob: 1.763534064025407e-05
Non Fraud prob: 1.7639673131952804e-05
Transaction: [16, 2, 5251, 1790789, 0, 1649736, 1790790, 76, 7, 36, 1790791, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 1.0260596837952107e-05
Non Fraud prob: 1.0

Transaction: [16, 54, 83, 1790891, 0, 1790892, 1, 30, 8, 50, 1790893, 65, 4, 18, 112, 31, 20, 3]
Fraud prob: 1.5795225067449792e-05
Non Fraud prob: 1.5870673223616905e-05
Transaction: [9, 2, 105, 1790895, 0, 1790896, 1, 218, 8, 150, 620087, 14, 12, 5, 123, 26, 61, 15]
Fraud prob: 1.2351668033261376e-05
Non Fraud prob: 1.2401982672827216e-05
Transaction: [17, 2, 28, 1500939, 0, 117, 1, 19, 7, 48, 118, 6, 4, 5, 59, 21, 33, 3]
Fraud prob: 0.0009555702202772154
Non Fraud prob: 0.0009555702202754432
Transaction: [9, 2, 25, 1790897, 0, 56, 1, 35, 7, 27, 842, 14, 12, 5, 13, 10, 20, 15]
Fraud prob: 0.9833931004779388
Non Fraud prob: 0.9833931004779431
Transaction: [9, 2, 588, 1790899, 0, 1790900, 1, 19, 7, 50, 1790901, 6, 4, 5, 13, 26, 11, 3]
Fraud prob: 1.2345236740429755e-05
Non Fraud prob: 1.2389718847316544e-05
Transaction: [9, 2, 7108, 1790903, 0, 1790904, 1, 19, 7, 23, 1790905, 6, 4, 5, 13, 10, 11, 3]
Fraud prob: 8.85417225049423e-06
Non Fraud prob: 8.899135309548e-06
Transaction: [16, 2

KeyboardInterrupt: 

In [ ]:
#end of test - 2nd step
end = time.time()

print(f"2nd Step Test: {end - begin}s")

In [64]:
#---------------------------------------------------density matrix------------------------------------------------------------#

#product of the matrices

#weight matrices
m1 = model.wv.vectors
m2 = model.syn1neg  #negative sampling
#m2 = model.syn1    #hierarchical-softmax 

mproduct = np.matmul(m1, m2.T)
mproduct.shape

(21938, 21938)

In [65]:
#apply softmax to obtain a matrix with conditional probabilities
conditional_probs = softmax(mproduct)

In [49]:
#confirm that softmax sum is 1
conditional_probs.sum()

0.99994487

In [50]:
#maximum value of the conditional probabilities
np.max(conditional_probs)

1.8367052e-07

In [51]:
#minimum value of the conditional probabilities
np.min(conditional_probs)

1e-45

In [78]:
#visualize the 1st 20 lines and the last 20 lines of the density matrix

#1st 20 lines
conditional_aux = conditional_probs[:20].copy()

#last 20 lines
conditional_aux = np.concatenate((conditional_aux, conditional_probs[-20:].copy()))

In [79]:
#save density matrix - after the product and the softmax
pd.DataFrame(conditional_aux).to_csv(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Transfers/Profiles/density_matrix.csv', header = None, index = False)

In [80]:
#shape of the compressed density matrix 
conditional_aux.shape

(40, 21938)

In [81]:
#-------------------------------------------------eigenvalues-----------------------------------------------------------------#

#singular eingenvalues of the square matrix - after the product and before softmax - w has the eigenvalues and v the eigenvectors
w, v = np.linalg.eig(mproduct)  #mproduct is an array of arrays

#separate real and imaginary parts of the singular eigenvalues
x = w.real  #array

y = w.imag  #array

In [82]:
#save eigenvalues in csv
df = pd.DataFrame({"Real" : x, "Imaginary" : y})

df.to_csv(r'C:/Users/BeatrizCarvalho/OneDrive - Closer Consultoria Lda/Documents/Entangled-Spaces/Transfers/Profiles/eigenvalues.csv', index = False)